# 1. Importing Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 2. Loading data

In [2]:
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# 3. Tokenizing the text to create sequence of words

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# 4. Creating input-output pairs

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 5. Padding input sequences to have equal length

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len, padding='pre'))

# 6. Splitting sequences into input and output

In [6]:
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

# 7. Converting the output to one-hot encode vectors

In [7]:
y = np.array(tf.keras.utils.to_categorical(y,num_classes=total_words))

# 8. Building Neural Network Architecture

In [8]:
model = Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# 9. Compile and Train the model

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=100,verbose=1)

Epoch 1/100
3010/3010 [==============================] - 45s 12ms/step - loss: 6.2291 - accuracy: 0.0794
Epoch 2/100
3010/3010 [==============================] - 27s 9ms/step - loss: 5.4824 - accuracy: 0.1255
Epoch 3/100
3010/3010 [==============================] - 27s 9ms/step - loss: 5.1088 - accuracy: 0.1471
Epoch 4/100
3010/3010 [==============================] - 26s 8ms/step - loss: 4.7820 - accuracy: 0.1661
Epoch 5/100
3010/3010 [==============================] - 25s 8ms/step - loss: 4.4807 - accuracy: 0.1838
Epoch 6/100
3010/3010 [==============================] - 25s 8ms/step - loss: 4.2009 - accuracy: 0.2046
Epoch 7/100
3010/3010 [==============================] - 25s 8ms/step - loss: 3.9321 - accuracy: 0.2279
Epoch 8/100
3010/3010 [==============================] - 27s 9ms/step - loss: 3.6779 - accuracy: 0.2558
Epoch 9/100
3010/3010 [==============================] - 25s 8ms/step - loss: 3.4362 - accuracy: 0.2909
Epoch 10/100
3010/3010 [==============================] - 25s 8

# 10. Saving the model

In [18]:
model.save('nwp_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 11. Generating the next word predictions

In [19]:
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model('nwp_model.h5')  # Replace 'my_model.h5' with the path to your saved model

seed_text = "I am excited to"
next_words = 2

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 22ms/step
I am excited to the altar
